# MSCI 434 Term Project

## 1. Import Packages and Licenses

In [1]:
%pip install gurobipy
import gurobipy as gp
from gurobipy import GRB as GRB

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [16]:
e = gp.Env(empty = True)
#e.setParam('HOSTNAME', 'v1040-wn-rt-b-69-164.campus-dynamic.uwaterloo.ca')
#e.setParam('HOSTID', 'dc5a3818')
#e.setParam('USERNAME', 'sarankan')
#e.setParam('KEY', 'GBQGVJSF')
#e.setParam('LICENSEID', 2397655)
e.start()

model = gp.Model(env = e)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-07-21


# 2. Setting up Gurobi 